In [1]:
import sys
print(sys.version)
import pandas
print('pandas',pandas.__version__)
import numpy
print('numpy',numpy.__version__)
from multiprocessing import Process
import os

3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]
pandas 0.23.4
numpy 1.13.3


https://docs.python.org/3.4/library/multiprocessing.html

In [15]:
def proc_info(some_str):
    print('arg =',some_str) # string that was passed in
    print('\nmodule name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def myfunc(a_name):
    """ this function calls another function """
    print('now in myfunc')
    print('process ID =',os.getpid())
    proc_info(a_name)
    print('end of myfunc')

before using multiprocessing, inspect the processes of a normal python execution

In [18]:
print('parent process:', os.getppid())
print('process ID =',os.getpid(),'\n')
myfunc('mary')

parent process: 7
process ID = 59 

now in myfunc
process ID = 59
arg = mary

module name: __main__
parent process: 7
process id: 59
end of myfunc


use multiprocessing module to launch a separate (serial) process

In [22]:
if __name__ == '__main__':
    proc_info('main code') # display info about this process
    print('\ncreate process')
    p = Process(target=myfunc, args=('bob',)) # from main, call a function
    print('\nstart process')
    p.start()
    print('\njoin')

    p.join()

arg = main code

module name: __main__
parent process: 7
process id: 59

create process

start process
now in myfunc
process ID = 751
arg = bob

join

module name: __main__
parent process: 59
process id: 751
end of myfunc


Now that we've explored having multiple processes, use the processes concurrently

https://docs.python.org/3.4/library/multiprocessing.html#using-a-pool-of-workers

In [4]:
from multiprocessing import Pool

In [5]:
def sq_fun(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(sq_fun, [1, 2, 3]))

[1, 4, 9]


Create random dataframe

https://stackoverflow.com/questions/32752292/how-to-create-a-data-frame-of-random-integers-with-pandas

In [6]:
df = pandas.DataFrame(
    numpy.random.randint(0,100,size=(100, 4)), 
    columns=list('ABCD')
    )
print(df.shape)
df.head()

(100, 4)


,A,B,C,D
0,47,13,97,23
1,18,78,75,69
2,93,92,6,13
3,99,63,1,65
4,62,52,94,96


http://www.racketracer.com/2016/07/06/pandas-in-parallel/

In [7]:
num_partitions = 7 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine

In [8]:
def parallelize_dataframe(df, func,num_cores):
    """
    Apply a function to chunks of a dataframe concurrently
    """
    df_split = numpy.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pandas.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [9]:
def multiply_columns(data):
    data['K'] = data['A'].apply(lambda x: x*2)
    return data
    
%time df_modified = parallelize_dataframe(df, multiply_columns,num_cores)

CPU times: user 60 ms, sys: 40 ms, total: 100 ms
Wall time: 302 ms


In [10]:
df.head()

,A,B,C,D
0,47,13,97,23
1,18,78,75,69
2,93,92,6,13
3,99,63,1,65
4,62,52,94,96


In [11]:
%time df['K'] = df['A'].apply(lambda x: x*2)

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 19.4 ms


Applying the function to chunks took much more time! 

`multiprocessing` has to serialize the data, send the serialized data to each thread, then apply the transformation, then send the data back to main